<a href="https://colab.research.google.com/github/samuel-mahembe/Data-Science-Python-Kickstart/blob/main/Script%2004%20-%20The%20Model%20Building%20Framework%20-%20Guided.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cloning GitHub Repo
!git clone https://github.com/chase-kusterer/BASD.git

# changing directory
import os
repo_name = '/content/BASD/'
os.chdir(repo_name)

# checking results
print(f"Current working directory changed to: {os.getcwd()}")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h1>The Model Building Framework</h1>
DAT-7470 - Business Analysis with Structured Data<br>
Sample Solution<br><br>

Written by Chase Kusterer - Faculty of Analytics <br>
Hult International Business School <br>
<a href="https://chase-kusterer.github.io/">https://chase-kusterer.github.io/</a> <br><br><br><br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

Our previous model building endeavors had one major drawback:<br><br>

<div align="center"><h4>The models were trained using all of the data.</h4></div><br>

This may not seem like a big deal, but modeling in this way can be very dangerous. Allowing an algorithm to see all of the data runs the risk of <strong>overfitting</strong>, or tailoring so closely to a dataset that the algorithm predicts poorly on new observations. Remember, the primary goal of building a model is to predict well on observations where the end result is unknown (i.e., new cases). Therefore, we need to set aside a some data before a model is trained (known as a <strong>testing</strong> or <strong>validation set</strong>). After training, the testing set will help us understand how the model predicts on new data.<br><br><br>
<strong>Some Things to Keep in Mind</strong><br>
1. Data exploration and feature engineering are always conducted on the full dataset.
2. Model adjustments should never be made on the testing set. The testing set should only be used to analyze a model's fit.<br>

<h4>Imports and Loading the Dataset</h4>
Run the following code to import packages and the feature-enhanced version of the dataset.

In [ ]:
# importing packages
import pandas as pd                                  # data science essentials
import matplotlib.pyplot as plt                      # data viz
import seaborn as sns                                # enhanced data viz
import statsmodels.formula.api as smf                # linear modeling
from sklearn.model_selection import train_test_split # train/test split
import sklearn.linear_model                          # faster linear modeling
import numpy as np                                   # mathematical essentials


# specifying the path and file name
file = './datasets/diamonds.xlsx'


# converting data types with a dictionary
data_types = {"channel" : str,
              "store"   : str}


# reading the file into Python
diamonds = pd.read_excel(io      = file,
                         dtype   = data_types,
                         usecols = range(1, 8)) # omitting Obs


# checking the file
diamonds.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part I: Preparing Categorical Data</h2><br>
Categorical features need to be converted into a series of yes or no questions, more formally known as <strong>binary matrix factorization</strong>. Also, one category needs to be removed to avoid issues with autocorrelation (the <em>k-1</em> rule). Note that we are intentionally omitting <em>store</em> in this part of the analysis as some retailers did not have sufficient sample sizes for stable vizualizations or model building.
<br><br>
Run the code below to instantiate the <em>categorical_boxplots</em> user-defined function.

In [ ]:
# defining a function for categorical boxplots
def categorical_boxplots(response, cat_var, data):
    """
	This function is designed to generate a boxplot for  can be used for categorical variables.
    Make sure matplotlib.pyplot and seaborn have been imported (as plt and sns).

    PARAMETERS
	----------
	response : str, response variable
	cat_var  : str, categorical variable
	data     : DataFrame of the response and categorical variables
	"""

    fig, ax = plt.subplots(figsize = (10, 8))

    sns.boxplot(x    = response,
                y    = cat_var,
                data = data)

    plt.suptitle("")
    plt.show()

<br>

In [ ]:
# calling the function for channel
categorical_boxplots(response = 'price',
					 cat_var  = 'channel',
					 data     = diamonds)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>a) Fill in the blanks in the code below to one hot encode <em>channel</em>.</strong>

In [ ]:
help(pd.get_dummies)

<br>

In [ ]:
# one hot encoding categorical variables
one_hot_channel = pd.____(data   = ____,
                          prefix = 'channel',
                          dtype  = int)


# joining with rest of dataset
diamonds2 = diamonds.join([one_hot_channel])


# checking results
diamonds2.head(n = 5)

<br>

In [ ]:
# renaming channels and storing as diamonds (not diamonds2)
diamonds2.rename(columns = {'channel_0': 'mall',
                            'channel_1': 'independent',
                            'channel_2': 'internet'},
                inplace = True)


# checking results
diamonds2.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Now, we need to decide which category to drop to avoid the <strong>dummy variable trap</strong>. In other words, the sum of all categories will always be one, making them perfectly collinear. Since most linear regression models also include an intercept, this creates a perfect linear relationship that makes the underlying math of models like OLS regression unsolvable.<br><br>
Choosing which category to remove tends to be less mathematical and more application oriented. This is because whatever is removed will act as the basis of interpretation for the categories included in the model. For example, if we remove the independent category, we are effectively setting its coefficient to zero. Thus, we would interpret a model's coefficient for the other channels as:
<br><br>
<div align="center"><h4>"Compared to independent jewelery stores, we can expect a price difference of <em>[coefficient value]</em> for diamonds being sold in <em>[channel]</em>."</h4></div><br>

We tend to use a frequency table (&nbsp;value_counts(&nbsp;)&nbsp;) to help arrive at an appropriate decision on which category to drop.

In [ ]:
# analyzing new feature counts
diamonds2[ ['mall', 'independent', 'internet'] ].sum()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<strong>b) Complete the code below to drop independent jewelery stores from the dataset.</strong>
Note that we are also dropping <em>channel</em> and <em>store</em> as they are currently not suitable for use in predicitive models.

In [ ]:
# dropping features as needed
diamonds = diamonds2.drop([_____,      # category to drop
                           'channel',  # already factorized
                           'store',    # not enough samples per store
                          ], axis = 1)


# checking results
diamonds.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part II: Training and Testing Sets</h2><br>

<h3>Organizing Data for Model Building</h3><br>
Before building models, we should organize our x-data into feature sets. This way, we can evaluate how different <strong>candidate models</strong> perform in predicting diamond engagement ring prices. Presently, we are focused on ordinary least squares regression. Ideally, we would apply each candidate model to more than model type. However, it's important to make sure we have a solid grasp of the modeling framework presented below before diving into more advanced concepts.

In [ ]:
## x-side ##
x_all  = diamonds.drop(labels = 'price', axis = 1)
x_base = diamonds[ ['carat', 'color', 'clarity', 'cut'] ]

## y-side ##
y_original = diamonds['price']

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Below we are selecting which feature sets to use. Notice that we do this for both the x- and y-sides of our model. Currently, we only have one version of the y-variable, but this will change as we advance in our modeling kowledge.
<br><br>
<strong>a) Complete the code below using <em>x_all</em>.</strong>

In [ ]:
# x-data
x_data = _____

# y_data
y_data = y_original

# train-test split
x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                    y_data,
                                                    test_size    = 0.35,
                                                    random_state = 702 )

<br>

In [ ]:
# checking results
print(f"""
Data Shapes
-----------
x_train: {x_train.shape}
y_train: {y_train.shape}

x_test:  {x_test.shape}
y_test:  {y_test.shape}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part III: Linear Regression Modeling</h2>

* We always build on the training set.
* We never touch the testing set.

In [ ]:
# naming the model
model_name = "Linear Regression"


# INSTANTIATING model object
model = sklearn.linear_model.LinearRegression()


# FITTING to training data
model_fit = model.fit(x_train, y_train)


# PREDICTING on new data
model_pred = model.predict(x_test)


# SCORING results (R-Square)
model_train_score = round(model.score(x_train, y_train), ndigits = 4)
model_test_score  = round(model.score(x_test, y_test), ndigits = 4)
model_gap         = round(model_train_score - model_test_score, ndigits = 4)


# displaying results
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

<br>
<h3>Extracting Coefficients</h3>

In [ ]:
# zipping each feature name to its coefficient
model_coefficients = zip(x_train.columns,
                         model.coef_.round(decimals = 4))


# setting up a placeholder list to store model features
coefficient_lst = [('intercept', model.intercept_.round(decimals = 4))]


# printing out each feature-coefficient pair one by one
for coefficient in model_coefficients:
    coefficient_lst.append(coefficient)


# checking the results
for pair in coefficient_lst:
    print(pair)

<br>
<h3>Model Summary</h3>

In [ ]:
# dynamically printing model summary
ols_model =  f"""\
Model Name:     {model_name}
Train_Score:    {model_train_score}
Test_Score:     {model_test_score}
Train-Test Gap: {model_gap}

Coefficents
-----------
{pd.DataFrame(data = coefficient_lst, columns = ["Feature", "Coefficient"])}"""

print(ols_model)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>How fit should a model be?</strong><br>
As a general heuristic, if the training and testing scores are within 0.05 of each other, the model has not been overfit. Don't worry if the testing score ends up higher than the training score. Some sources claim that in such situations a model is underfit, but this is a general misconception that is beyond the scope of this course.<br><br>

<h2>Exploring the Model Building Framework</h2><br>
Let's explore each component of the model building framework.

In [ ]:
help(sklearn.linear_model.LinearRegression)

<br>
<h3>Instantiate</h3>
This creates a blueprint of the model, just like with user-defined functions.

In [ ]:
# this is a blueprint
model = sklearn.linear_model.LinearRegression()


# printing blueprint
print(model)

<br><h3>Fit</h3>
Runs the data through the model, creating regression metrics such as R-Square and coefficients.

In [ ]:
# model is created from blueprint and data
model_fit = model.fit(x_train, y_train)


# printing model attributes
print(f"""
Intercept
---------
{round(model_fit.intercept_, ndigits = 2)}


Coefficients
------------
{model_fit.coef_.round(decimals = 2)}


Total X-Features
----------------
{model_fit.n_features_in_}
""")

<br><h3>Predict</h3>
Uses the fitted model to predict on the testing set. This helps us understand the stability of the model.

In [ ]:
# applying model to validation set
model_pred = model.predict(x_test)


# printing predictions (validation set)
print(model_pred.round(decimals = 0).astype(dtype = int))

<br>

In [ ]:
## Residual Analysis ##

# organizing residuals
model_residuals = {
    "True"            : y_test,
    "Predicted"       : model_pred.round(decimals = 0).astype(dtype = int)
}


# converting residuals into df
resid_df = pd.DataFrame(data = model_residuals)


# checking results
resid_df.head(n = 5)

<br><br>
<strong>a)</strong> Complete the code to develop a residual plot from <em>resid_df</em>. Plot <em>Predicted</em> on the x-axis and <em>True</em> on the y-axis.

In [ ]:
# developing a residual plot
sns.residplot(data        = _____,
              x           = _____,
              y           = _____,
              lowess      = True,
              color       = 'blue',
              scatter_kws = {'alpha': 0.3},   # data point transparency
              line_kws    = {'color': 'red'}) # line color


# title and axis labels
plt.title(label   = "Residual Plot - Full Model")
plt.xlabel(xlabel = "Predicted Sale Price")
plt.ylabel(ylabel = "Residual Sale Price")


# layout and rendering visual
plt.tight_layout()
plt.show()

<br>
<h3>Score</h3>
Quantifies the quality of predictions based on a scoring metric (in this case, R-Square). More information on scoring metrics <a href="https://scikit-learn.org/stable/modules/model_evaluation.html">can be found here</a>.

In [ ]:
# checking overall predictive quality
print('Training Score:', round(model.score(x_train, y_train), ndigits = 4))
print('Testing  Score:', round(model.score(x_test, y_test),   ndigits = 4))
print('Train-Test Gap:', model_gap)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part IV: Practice Time!</h2><br>
Run this script again using different sets of X-data.<br>

* Which feature set results in the highest R-Square value?
* Which one results in the lowest train-test gap?
* Are there any feature sets that flatten out the lowess estimator in the residual plot?

<br><br><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

~~~

  _    _                           __  __           _      _ _             _
 | |  | |                         |  \/  |         | |    | (_)           | |
 | |__| | __ _ _ __  _ __  _   _  | \  / | ___   __| | ___| |_ _ __   __ _| |
 |  __  |/ _` | '_ \| '_ \| | | | | |\/| |/ _ \ / _` |/ _ \ | | '_ \ / _` | |
 | |  | | (_| | |_) | |_) | |_| | | |  | | (_) | (_| |  __/ | | | | | (_| |_|
 |_|  |_|\__,_| .__/| .__/ \__, | |_|  |_|\___/ \__,_|\___|_|_|_| |_|\__, (_)
              | |   | |     __/ |                                     __/ |  
              |_|   |_|    |___/                                     |___/   
                                                                 
~~~

<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Bonus I: Using Statsmodels for p-values</h2>

Although p-values are not a concept in scikit-learn, we can apply our knowledge of statsmodels to generate them. Note that p-values are only applicable to a small handful of models. For our purposes, they may come in handy for OLS regression. Run the code below if you would like to generate an output that includes p-values for all features.

In [ ]:
# Step 1: INSTANTIATE a model object
lm = smf.ols(formula =  """price ~ carat +
                                   color +
                                   clarity +
                                   cut +
                                   mall +
                                   internet""",
            data = diamonds)


# Step 2: FIT the data into the model object
results = lm.fit()


# Step 3: analyze the SUMMARY output
print(results.summary())

<br>
<h2>Bonus II: Saving the Dataset to Excel</h2>

Run the following code to learn about the <em>to_excel(&nbsp;)</em> method. Then, save the prepared dataset as an Excel file.

In [ ]:
help(diamonds.to_excel)

<br>

In [ ]:
# storing the dataset as an Excel file
diamonds.to_excel(excel_writer = './datasets/diamonds_prepped.xlsx',
                  index = False)

<br>